In [1]:
# Chargement des librairies
import requests
import pandas as pd
import json

from src.config import RAW_DATA_DIR, JOLPICA_BASE_URL


In [15]:
def fetch_page(endpoint, limit=100, offset=0):
    """
    Effectue une requete HTTP GET vers l'API Jolpica.
    
    Arguments:
    ----------
    endpoint : str
        Ressource cible (ex: 'drivers', 'results').
    limit : int
        Nombre maximum d'enregistrements par page (defaut: 100).
    offset : int
        Indice de depart pour la pagination.
        
    Retourne:
    ---------
    dict or None : Le contenu JSON de la reponse ou None en cas d'erreur.
    """
    url = f"{JOLPICA_BASE_URL}/{endpoint}.json?limit={limit}&offset={offset}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur reseau sur {url} : {str(e)}")
        return None

In [16]:
def get_all(endpoint):
    """
    Recupere l'integralite des donnees d'un endpoint via une boucle de pagination.
    
    Cette fonction automatise la navigation dans la structure MRData
    et concatene les resultats dans une liste unique.
    """
    all_results = []
    limit = 100
    offset = 0
    
    # Mapping des cles specifiques au standard Ergast
    table_map = {
        "drivers": "DriverTable",
        "circuits": "CircuitTable",
        "constructors": "ConstructorTable",
        "results": "RaceTable"
    }
    
    table_key = table_map.get(endpoint, f"{endpoint.capitalize()}Table")
    list_key = endpoint.capitalize()
    
    while True:
        data = fetch_page(endpoint, limit, offset)
        if not data:
            break
            
        mr_data = data['MRData']
        new_items = mr_data[table_key][list_key]
        all_results.extend(new_items)
        
        total = int(mr_data['total'])
        print(f"Collecte {endpoint} : {len(all_results)} / {total}")
        
        if len(all_results) >= total:
            break
            
        offset += limit
        
    return all_results

In [17]:
drivers = get_all("drivers")

Collecte drivers : 100 / 874
Collecte drivers : 200 / 874
Collecte drivers : 300 / 874
Collecte drivers : 400 / 874
Collecte drivers : 500 / 874
Collecte drivers : 600 / 874
Collecte drivers : 700 / 874
Collecte drivers : 800 / 874
Collecte drivers : 874 / 874


In [ ]:
def save_raw_json(data, resource_name):
    # On cree un nom de fichier simple : ex data/raw/drivers.json
    file_path = RAW_DATA_DIR / f"{resource_name}.json"
    
    with open(file_path, "w", encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    print(f"Succès : Fichier brut sauvé dans {file_path}")
    return file_path